In [1]:
pip install opencv-python opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install scikit-image

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
pip install scikit-learn

In [2]:
pip install numpy opencv-python pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tensorflow

In [ ]:
pip install torch torchvision

In [1]:
import cv2
import numpy as np

# Input video file path
video_path = r"D:\\Sem-4\\Project\\Datasets\\VISEM-Tracking - Datasets from Paper 2\\VISEM_Tracking_Train_v4\\Train\\11\\11.mp4"

# Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Scaling factor to resize the video frames
scale_factor = 0.5

# Overlay text
font = cv2.FONT_HERSHEY_SIMPLEX
org_text = (10, 30)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Resize the frame
    frame_resized = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # 1. Grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # 2. Background Subtraction
    foreground_mask = bg_subtractor.apply(frame_resized)

    # 3. Thresholding
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # 4. Morphological Operations
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)  # Opening
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # Closing

    # 5. Edge Detection
    edges = cv2.Canny(gray, 50, 150)

    # 6. Blurring
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 7. Contour Detection
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_frame = frame_resized.copy()
    cv2.drawContours(contour_frame, contours, -1, (0, 255, 0), 2)

    # Combine all images into a single visualization with labels
    top_row = np.hstack([
        cv2.putText(frame_resized.copy(), 'Original', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), 'Grayscale', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR), 'Foreground', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    middle_row = np.hstack([
        cv2.putText(cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR), 'Threshold', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR), 'Opened', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR), 'Closed', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    bottom_row = np.hstack([
        cv2.putText(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 'Edges', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR), 'Blurred', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(contour_frame, 'Contours', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])

    # Stack the combined images
    combined = np.vstack([top_row, middle_row, bottom_row])

    # Display the combined visualization with file name and quit instructions
    file_name = video_path.split("\\")[-1]
    cv2.putText(combined, f'File: {file_name}', (10, 50), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(combined, 'Press "q" to quit', (10, 80), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Classic Methods Visualization', combined)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Properly release the video and close the window
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
import cv2
import numpy as np
import math
from scipy.optimize import linear_sum_assignment

# Input video file path
video_path = r"D:\Sem-4\Project\Papers\VISEM - Dataset from Paper 1 MP4 format\1_09.09.02_SSW.mp4"

# Load the video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Tracking parameters
tracking_data = {}  # Stores sperm ID and centroid
next_sperm_id = 1
max_distance = 50  # Maximum distance to consider the same sperm
immotile_threshold = 5
cluster_threshold = 20
history_limit = 10
lost_frames_threshold = 10  # Number of frames to keep tracking lost sperms

# Helper function to calculate Euclidean distance
def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Preprocess the frame
    fgmask = bg_subtractor.apply(frame)
    _, binary = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Find contours
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Current frame's detected centroids
    current_centroids = []
    for contour in contours:
        if cv2.contourArea(contour) < 50:  # Ignore small noise
            continue
        x, y, w, h = cv2.boundingRect(contour)
        cx, cy = int(x + w / 2), int(y + h / 2)  # Centroid of the bounding box
        current_centroids.append((cx, cy))

    # Update tracking using Hungarian Algorithm
    if tracking_data:
        existing_ids = list(tracking_data.keys())
        existing_centroids = [tracking_data[sperm_id]['centroid'] for sperm_id in existing_ids]

        # Compute cost matrix based on Euclidean distance
        cost_matrix = np.zeros((len(existing_centroids), len(current_centroids)))
        for i, ec in enumerate(existing_centroids):
            for j, cc in enumerate(current_centroids):
                cost_matrix[i, j] = euclidean_distance(ec, cc)

        # Solve the assignment problem
        row_ind, col_ind = linear_sum_assignment(cost_matrix)

        # Update tracking data
        unmatched_existing = set(range(len(existing_centroids)))
        unmatched_current = set(range(len(current_centroids)))
        for r, c in zip(row_ind, col_ind):
            if cost_matrix[r, c] < max_distance:
                sperm_id = existing_ids[r]
                tracking_data[sperm_id]['centroid'] = current_centroids[c]
                tracking_data[sperm_id]['lost_frames'] = 0
                tracking_data[sperm_id]['history'].append(current_centroids[c])
                unmatched_existing.remove(r)
                unmatched_current.remove(c)

        # Mark unmatched existing IDs as lost
        for r in unmatched_existing:
            sperm_id = existing_ids[r]
            tracking_data[sperm_id]['lost_frames'] += 1

        # Add new IDs for unmatched current centroids
        for c in unmatched_current:
            tracking_data[next_sperm_id] = {
                'centroid': current_centroids[c],
                'history': [current_centroids[c]],
                'immotile': False,
                'lost_frames': 0,
            }
            next_sperm_id += 1
    else:
        # Initialize tracking with the first frame
        for centroid in current_centroids:
            tracking_data[next_sperm_id] = {
                'centroid': centroid,
                'history': [centroid],
                'immotile': False,
                'lost_frames': 0,
            }
            next_sperm_id += 1

    # Remove lost IDs
    tracking_data = {id_: data for id_, data in tracking_data.items() if data['lost_frames'] < lost_frames_threshold}

    # Check immotile status
    for sperm_id, data in tracking_data.items():
        if len(data['history']) >= history_limit:
            start_point = data['history'][-history_limit]
            end_point = data['centroid']
            if euclidean_distance(start_point, end_point) < immotile_threshold:
                tracking_data[sperm_id]['immotile'] = True
            else:
                tracking_data[sperm_id]['immotile'] = False

    # Visualization
    for sperm_id, data in tracking_data.items():
        cx, cy = data['centroid']
        x, y, w, h = cv2.boundingRect(np.array([[cx, cy]]))

        # Check for clusters
        is_cluster = sum(1 for other_id, other_data in tracking_data.items()
                         if other_id != sperm_id and euclidean_distance(data['centroid'], other_data['centroid']) < cluster_threshold) > 0

        if is_cluster:
            color = (0, 0, 255)  # Red for clusters
        elif data['immotile']:
            color = (255, 0, 0)  # Blue for immotile
        else:
            color = (0, 255, 0)  # Green for active sperms

        # Draw bounding box and label with sperm_tracking_id
        cv2.rectangle(frame, (cx - w // 2, cy - h // 2), (cx + w // 2, cy + h // 2), color, 2)
        cv2.putText(frame, f"ID: {sperm_id}", (cx - 10, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Display the frame
    cv2.imshow('Sperm Tracking with Persistent IDs', frame)

    if cv2.waitKey(30) & 0xFF == 27:  # Press 'ESC' to quit
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import math
from scipy.optimize import linear_sum_assignment
import csv

# Input video file path
video_path = r"D:\Sem-4\Project\Papers\VISEM - Dataset from Paper 1 MP4 format\1_09.09.02_SSW.mp4"

# Load the video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Tracking parameters
tracking_data = {}  # Stores sperm ID, centroid, history, etc.
next_sperm_id = 1
max_distance = 50  # Maximum distance to consider the same sperm
immotile_threshold = 5  # Movement threshold for immotile status
history_limit = 10
lost_frames_threshold = 10  # Number of frames to keep tracking lost sperms
frame_rate = 30  # Frame rate of the video (fps)

# Output CSV file
output_csv = "sperm_tracking_status.csv"
with open(output_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Frame", "Sperm ID", "Motility", "Velocity (pixels/frame)", "Trajectory (x, y)"])

# Helper function to calculate Euclidean distance
def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Helper function to classify motility
def classify_motility(history, velocity):
    if velocity < immotile_threshold:
        return "Immotile"
    elif len(history) >= history_limit:
        # Check overall directionality (progressive vs. non-progressive)
        start_point = history[0]
        end_point = history[-1]
        net_distance = euclidean_distance(start_point, end_point)
        if net_distance > len(history) * immotile_threshold:
            return "Progressive Motile"
        else:
            return "Non-Progressive Motile"
    else:
        return "Unknown"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Preprocess the frame
    fgmask = bg_subtractor.apply(frame)
    _, binary = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Find contours
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Current frame's detected centroids
    current_centroids = []
    for contour in contours:
        if cv2.contourArea(contour) < 50:  # Ignore small noise
            continue
        x, y, w, h = cv2.boundingRect(contour)
        cx, cy = int(x + w / 2), int(y + h / 2)  # Centroid of the bounding box
        current_centroids.append((cx, cy))

    # Update tracking using Hungarian Algorithm
    if tracking_data:
        existing_ids = list(tracking_data.keys())
        existing_centroids = [tracking_data[sperm_id]['centroid'] for sperm_id in existing_ids]

        # Compute cost matrix based on Euclidean distance
        cost_matrix = np.zeros((len(existing_centroids), len(current_centroids)))
        for i, ec in enumerate(existing_centroids):
            for j, cc in enumerate(current_centroids):
                cost_matrix[i, j] = euclidean_distance(ec, cc)

        # Solve the assignment problem
        row_ind, col_ind = linear_sum_assignment(cost_matrix)

        # Update tracking data
        unmatched_existing = set(range(len(existing_centroids)))
        unmatched_current = set(range(len(current_centroids)))
        for r, c in zip(row_ind, col_ind):
            if cost_matrix[r, c] < max_distance:
                sperm_id = existing_ids[r]
                tracking_data[sperm_id]['centroid'] = current_centroids[c]
                tracking_data[sperm_id]['lost_frames'] = 0
                tracking_data[sperm_id]['history'].append(current_centroids[c])

                # Calculate velocity
                prev_position = tracking_data[sperm_id]['history'][-2] if len(tracking_data[sperm_id]['history']) > 1 else current_centroids[c]
                tracking_data[sperm_id]['velocity'] = euclidean_distance(prev_position, current_centroids[c]) * frame_rate
                unmatched_existing.remove(r)
                unmatched_current.remove(c)

        # Mark unmatched existing IDs as lost
        for r in unmatched_existing:
            sperm_id = existing_ids[r]
            tracking_data[sperm_id]['lost_frames'] += 1

        # Add new IDs for unmatched current centroids
        for c in unmatched_current:
            tracking_data[next_sperm_id] = {
                'centroid': current_centroids[c],
                'history': [current_centroids[c]],
                'velocity': 0,
                'lost_frames': 0,
            }
            next_sperm_id += 1
    else:
        # Initialize tracking with the first frame
        for centroid in current_centroids:
            tracking_data[next_sperm_id] = {
                'centroid': centroid,
                'history': [centroid],
                'velocity': 0,
                'lost_frames': 0,
            }
            next_sperm_id += 1

    # Remove lost IDs
    tracking_data = {id_: data for id_, data in tracking_data.items() if data['lost_frames'] < lost_frames_threshold}

    # Log data to CSV and draw on the frame
    with open(output_csv, "a", newline="") as f:
        writer = csv.writer(f)
        for sperm_id, data in tracking_data.items():
            motility = classify_motility(data['history'], data['velocity'])
            trajectory = data['history']
            centroid = data['centroid']
            velocity = data['velocity']

            # Write data to the CSV file
            writer.writerow([cap.get(cv2.CAP_PROP_POS_FRAMES), sperm_id, motility, velocity, trajectory])

            # Visualize
            color = (0, 255, 0) if motility == "Progressive Motile" else (255, 0, 0) if motility == "Immotile" else (0, 0, 255)
            cv2.circle(frame, centroid, 5, color, -1)
            cv2.putText(frame, f"ID:{sperm_id}", (centroid[0] + 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Display the frame
    cv2.imshow('Sperm Tracking', frame)

    if cv2.waitKey(30) & 0xFF == 27:  # Press 'ESC' to quit
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade opencv-python opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
#To save the processed video
import cv2
import numpy as np

# Input video file path
video_path = r"D:\\Sem-4\\Project\\Datasets\\VISEM-Tracking - Datasets from Paper 2\\VISEM_Tracking_Train_v4\\Train\\11\\11.mp4"

# Output video file path
output_path = r"D:\\Sem-4\\Project\\Processed_Output.avi"

# Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for the output video

# Scale down the resolution for processing
scale_factor = 0.5
output_width = int(frame_width * scale_factor)
output_height = int(frame_height * scale_factor)

# Initialize VideoWriter for saving the processed video
out = cv2.VideoWriter(output_path, fourcc, fps, (output_width * 3, output_height * 3))  # 3x3 grid

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Overlay text
font = cv2.FONT_HERSHEY_SIMPLEX

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Resize the frame
    frame_resized = cv2.resize(frame, (output_width, output_height), interpolation=cv2.INTER_AREA)

    # 1. Grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # 2. Background Subtraction
    foreground_mask = bg_subtractor.apply(frame_resized)

    # 3. Thresholding
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # 4. Morphological Operations
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)  # Opening
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # Closing

    # 5. Edge Detection
    edges = cv2.Canny(gray, 50, 150)

    # 6. Blurring
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 7. Contour Detection
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_frame = frame_resized.copy()
    cv2.drawContours(contour_frame, contours, -1, (0, 255, 0), 2)

    # Combine all images into a single visualization
    top_row = np.hstack([
        cv2.putText(frame_resized.copy(), 'Original', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), 'Grayscale', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR), 'Foreground', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    middle_row = np.hstack([
        cv2.putText(cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR), 'Threshold', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR), 'Opened', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR), 'Closed', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    bottom_row = np.hstack([
        cv2.putText(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 'Edges', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR), 'Blurred', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(contour_frame, 'Contours', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])

    # Stack the combined images
    combined = np.vstack([top_row, middle_row, bottom_row])

    # Write to the output video
    out.write(combined)

# Properly release the video and writer
cap.release()
out.release()
print(f"Processing complete. Saved to {output_path}.")


End of video.
Processing complete. Saved to D:\\Sem-4\\Project\\Processed_Output.avi.


In [6]:
import cv2
import numpy as np

# Input video file path
video_path = r"D:\\Sem-4\\Project\\Datasets\\VISEM-Tracking - Datasets from Paper 2\\VISEM_Tracking_Train_v4\\Train\\11\\11.mp4"

# Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Scaling factor to resize the video frames
scale_factor = 0.5

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Overlay text
font = cv2.FONT_HERSHEY_SIMPLEX

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Resize the frame
    frame_resized = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # 1. Grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # 2. Background Subtraction
    foreground_mask = bg_subtractor.apply(frame_resized)

    # 3. Thresholding
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # 4. Morphological Operations
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)  # Opening
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # Closing

    # 5. Edge Detection
    edges = cv2.Canny(gray, 50, 150)

    # 6. Blurring
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 7. Contour Detection
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_frame = frame_resized.copy()
    cv2.drawContours(contour_frame, contours, -1, (0, 255, 0), 2)

    # Combine all images into a single visualization
    top_row = np.hstack([
        cv2.putText(frame_resized.copy(), 'Original', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), 'Grayscale', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR), 'Foreground', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    middle_row = np.hstack([
        cv2.putText(cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR), 'Threshold', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR), 'Opened', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR), 'Closed', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    bottom_row = np.hstack([
        cv2.putText(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 'Edges', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR), 'Blurred', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(contour_frame, 'Contours', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])

    # Stack the combined images
    combined = np.vstack([top_row, middle_row, bottom_row])

    # Display the combined visualization
    cv2.imshow('Classic Methods Visualization', combined)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Properly release the video and close the window
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
